# Tutorial - The Preamble

**Author(s):** Santosh Philip

We am using this tutorial/documentation as a way of identifying alformed API and gaps in the software. When eppy was developed, writing the user documentation forced us to rewrite the software. This documentation is written with the same intent. It is a little easier in the case of `eppy3000`, since we are trying to mimic `eppy` using it as a starting point.

Any places where the word `TODO` occurs is where we find some recoding has to be done. An issue has to be opened and then resolved.

## Start with Eppy

We are now at E+ version 9.3. If you install E+ version 9.3, you will find only one epJSON file in the installation. All the files are IDF files. So how do we work with and explore eppy300 and epJSON ?

We will have to read the IDF file, convert it to epJSON and then work with it. The key is to do the conversion from IDF to epJSON. There are two ways of doing this:

1. import eppy. Read the IDF using eppy and then do the conversion
2. Use eppy3000 to directly convert IDF to epJSON. Don't use eppy at all

## Convert to epJSON *with* eppy

In [1]:
# you would normaly install eppy by doing
# python setup.py install
# or
# pip install eppy
# or
# easy_install eppy

# if you have not done so, uncomment the following three lines
import sys
# pathnameto_eppy = 'c:/eppy3000'
pathnameto_eppy3000 = '../'
sys.path.append(pathnameto_eppy3000)

import eppy

In [2]:
fname = "../eppy3000/resources/snippets/V9_3/smallfile.idf"
idf = eppy.openidf(fname)
idf.printidf()


VERSION,
    9.3;                      !- Version Identifier

SIMULATIONCONTROL,
    Yes,                      !- Do Zone Sizing Calculation
    Yes,                      !- Do System Sizing Calculation
    Yes,                      !- Do Plant Sizing Calculation
    No,                       !- Run Simulation for Sizing Periods
    Yes;                      !- Run Simulation for Weather File Run Periods

BUILDING,
    Empire State Building,    !- Name
    30,                       !- North Axis
    City,                     !- Terrain
    0.04,                     !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullExterior,             !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    6;                        !- Minimum Number of Warmup Days

SITE:LOCATION,
    CHICAGO_IL_USA TMY2-94846,    !- Name
    41.78,                    !- Latitude
    -87.75,                   !- Longit

Looks good. Now we need to convert it

In [3]:
import eppy3000.oldeppy as oldeppy
epschema = '../eppy3000/resources/schema/V9_3/Energy+.schema.epJSON'
epj = oldeppy.idf2epj(idf, open(epschema, 'r'))

#TODO: would be nice to have the code find the schema file in the E+ installation.
# Like how eppy2000.openidf() does

Let us print this `epj` and see what it looks like

In [4]:
print(epj)


Version                                          !-  EP_KEY
            Version 1                            !-  EPJOBJECT_NAME
            9.3                                  !-  version_identifier
            1                                    !-  idf_order

SimulationControl                                !-  EP_KEY
            SimulationControl 1                  !-  EPJOBJECT_NAME
            Yes                                  !-  do_zone_sizing_calculation
            Yes                                  !-  do_system_sizing_calculation
            Yes                                  !-  do_plant_sizing_calculation
            No                                   !-  run_simulation_for_sizing_periods
            Yes                                  !-  run_simulation_for_weather_file_run_periods
            2                                    !-  idf_order

Building                                         !-  EP_KEY
            Empire State Building                !-  EPJ

Thats weird . I looks like the idf file. Shouldn't it look like the JSON ?

Good point. The actual JSON in `epj` looks different. Let us take a look at that

In [5]:
 print(epj.jsonstr())

{
    "Version": {
        "Version 1": {
            "version_identifier": "9.3",
            "idf_order": 1
        }
    },
    "SimulationControl": {
        "SimulationControl 1": {
            "do_zone_sizing_calculation": "Yes",
            "do_system_sizing_calculation": "Yes",
            "do_plant_sizing_calculation": "Yes",
            "run_simulation_for_sizing_periods": "No",
            "run_simulation_for_weather_file_run_periods": "Yes",
            "idf_order": 2
        }
    },
    "Building": {
        "Empire State Building": {
            "north_axis": 30,
            "terrain": "City",
            "loads_convergence_tolerance_value": 0.04,
            "temperature_convergence_tolerance_value": 0.4,
            "solar_distribution": "FullExterior",
            "maximum_number_of_warmup_days": 25,
            "minimum_number_of_warmup_days": 6,
            "idf_order": 3
        }
    },
    "Site:Location": {
        "CHICAGO_IL_USA TMY2-94846": {
            "lat

So why are you printing it to look like an IDF file. 

Couple of reasons:

1. It is was easier to do the development with epj expressed in the IDF format.
    - Note that there are some extra fields in the idf format that are not there in the JSON format.
        - One of them is `EP_KEY`
        - the other is `EPJOBJECT_NAME`
    - These fields are used internally by eppy3000
    - epJSON allows you to add your own fields. A very useful attribute
    - Any field name that starts with `EP` will be used internally by eppy3000 and will be displayed
    - Any field name that starts with `eppy_` will be used internally by eppy3000, but will not be displayed in the print (these are hidden fields, that the user does not need to see)
2. The second reason, is that the JSON fromat is not as **human readable** as it claims to be. At least not for most humans :-). 
    - as an example see this link https://marianoguerra.github.io/json.human.js/
    - Presenting it in the IDF fromat makes it clearer
   
It is not clear if this is the best pathway to take. But it is useful for the moment. So we are going to stick with it until something better comes along

#### EPJ schema

With the IDF file, we also have the IDD file that holds the structure of the IDF file. In eppy we can access IDD file. What happens in epJSON file in EPJ

Let us get to the fieldnames of `Building` object as seen by the schema

In [6]:
# building_fieldnames = epj.epschema.epschemaobjects['Building'].fieldnames()
#
# There is bug that does not capture the epschema 
# happens when idf2epj is used 
# this is a TODO: open an issue on it


## Convert to epJSON *without* eppy

In [7]:
import eppy3000.idfjsonconverter as idfjsonconverter

fname = "../eppy3000/resources/snippets/V9_3/smallfile.idf"
schemapath = "../eppy3000/resources/schema/V9_3/Energy+.schema.epJSON"
idfhandle = open(fname, 'r')

jsonstr = idfjsonconverter.idf2json(idfhandle, open(schemapath, 'r'))
print(jsonstr)

{
  "Version": {
    "Version 1": {
      "version_identifier": "9.3",
      "idf_order": 1
    }
  },
  "SimulationControl": {
    "SimulationControl 1": {
      "do_zone_sizing_calculation": "Yes",
      "do_system_sizing_calculation": "Yes",
      "do_plant_sizing_calculation": "Yes",
      "run_simulation_for_sizing_periods": "No",
      "run_simulation_for_weather_file_run_periods": "Yes",
      "idf_order": 2
    }
  },
  "Building": {
    "Empire State Building": {
      "north_axis": 30,
      "terrain": "City",
      "loads_convergence_tolerance_value": 0.04,
      "temperature_convergence_tolerance_value": 0.4,
      "solar_distribution": "FullExterior",
      "maximum_number_of_warmup_days": 25,
      "minimum_number_of_warmup_days": 6,
      "idf_order": 3
    }
  },
  "Site:Location": {
    "CHICAGO_IL_USA TMY2-94846": {
      "latitude": 41.78,
      "longitude": -87.75,
      "time_zone": -6,
      "elevation": 190,
      "idf_order": 4
    }
  }
}


We get a JSON string. We need to read this string into eppy3000

In [8]:
from io import StringIO
from eppy3000.modelmaker import EPJ

epj = EPJ(StringIO(jsonstr))

In [9]:
print(epj)


Version                                          !-  EP_KEY
            Version 1                            !-  EPJOBJECT_NAME
            9.3                                  !-  version_identifier
            1                                    !-  idf_order

SimulationControl                                !-  EP_KEY
            SimulationControl 1                  !-  EPJOBJECT_NAME
            Yes                                  !-  do_zone_sizing_calculation
            Yes                                  !-  do_system_sizing_calculation
            Yes                                  !-  do_plant_sizing_calculation
            No                                   !-  run_simulation_for_sizing_periods
            Yes                                  !-  run_simulation_for_weather_file_run_periods
            2                                    !-  idf_order

Building                                         !-  EP_KEY
            Empire State Building                !-  EPJ

## Opening a file without the schema (IDD)

Eppy3000 lets you open the `epJSON` as `EPJ` file without refering to the `schema.epJSON`. The code in the previous section does exactly that. A lot of functionality is avaliable without the schema. There are use cases for this, where you you want to quickly make some changes to `epj` and don't have access to the schema.

TODO: It is not explicit in the API whether the schema is read or not. One way to make it explicit is to force the user to enter the arg for the schema. If the arg is None: no schema is read; else: the schema is read